<a href="https://colab.research.google.com/github/MithilaCoomaraswamy/DSGP/blob/Exercise-Recommender/Exercise_Rec_KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix

# **Data Preprocessing**

In [2]:
# Load the dataset
data = pd.read_csv('Exercise dataset.csv')

In [3]:
data.head()

,Weight,Height,BMI,Age,BMIcase,Exercise Intensity,Gym Exercise,Home Exercise
0,92.085190,1.760250,29.719488,59,over weight,5,Treadmill Walking with Intervals; Moderate Wei...,Modified Burpees; Step-Ups on a Sturdy Chair o...
1,61.089124,1.595499,23.997776,25,normal,4,Moderate Weight Training (Full Body Workout); ...,Jump Rope; High Knees; Planks; Squats and Lung...
2,82.454037,1.816538,24.987499,50,normal,4,Moderate Weight Training (Full Body Workout); ...,Jump Rope; High Knees; Planks; Squats and Lung...
3,101.713306,1.790696,31.720047,62,obese,6,"Low-Impact Cardio (Elliptical, Cycling); Seate...",Chair Squats; Modified Plank; Seated Leg Lifts...
4,62.347337,1.543259,26.178219,35,over weight,5,Treadmill Walking with Intervals; Moderate Wei...,Modified Burpees; Step-Ups on a Sturdy Chair o...


In [11]:

# Indentify each numerical data range
data.describe().loc[['min','max']]

,Weight,Height,BMI,Age,Exercise Intensity,BMIcase_encoded
min,50.002978,1.390171,15.106978,18.0,1.0,0.0
max,109.998871,1.829867,34.997098,65.0,7.0,6.0


In [12]:
# Identifing the information of the data
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2492 entries, 0 to 2491
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Weight              2492 non-null   float64
 1   Height              2492 non-null   float64
 2   BMI                 2492 non-null   float64
 3   Age                 2492 non-null   int64  
 4   BMIcase             2492 non-null   object 
 5   Exercise Intensity  2492 non-null   int64  
 6   Gym Exercise        2492 non-null   object 
 7   Home Exercise       2492 non-null   object 
 8   BMIcase_encoded     2492 non-null   int64  
dtypes: float64(3), int64(3), object(3)
memory usage: 175.3+ KB


In [13]:
# Value counts in each categorical data

# Making a dictionary for store categorical data
value_counts_dict = {}

# Value count each categorical data by looping and save it in a dictionary

for col in data.columns:
    if data[col].dtype == 'object' or data[col].dtype == 'category':
        value_counts_dict[col] = data[col].value_counts()

# Print each categorical data by looping

for col, value_counts in value_counts_dict.items():
    print(f"Value counts for column '{col}':\n{value_counts}\n")

Value counts for column 'BMIcase':
BMIcase
severe obese         784
over weight          575
normal               550
obese                483
mild thinness         60
moderate thinness     27
sever thinness        13
Name: count, dtype: int64

Value counts for column 'Gym Exercise':
Gym Exercise
Seated Cardio Machines (Rowing, Recumbent Bike); Water-Based Aerobics; Seated Strength Machines (Low Weights); Treadmill Walking at a Slow Pace                                   784
Treadmill Walking with Intervals; Moderate Weightlifting (Upper and Lower Body); Stationary Biking (High Resistance Intervals); Resistance Circuit Training                       575
Moderate Weight Training (Full Body Workout); Elliptical (Moderate Intensity); Stair Climber (Low to Moderate Intensity); Rowing Machine (High Intensity)                         550
Low-Impact Cardio (Elliptical, Cycling); Seated Weight Exercises; Resistance Bands and Cable Exercises; Rowing Machine (Low Resistance)                     

In [14]:
# Print each unique categorical data

unique_dict = {}

 # Search for unique categorical data by looping and save it in a dictionary

for col in data.columns:
    if data[col].dtype == 'object' or data[col].dtype == 'category':
        unique_dict[col] = data[col].unique()

# Print each unique categorical data by looping

for col, unique in unique_dict.items():
    print(f"'{col}' unique:\n{unique}\n")

'BMIcase' unique:
['over weight' 'normal' 'obese' 'severe obese' 'mild thinness'
 'moderate thinness' 'sever thinness']

'Gym Exercise' unique:
['Treadmill Walking with Intervals; Moderate Weightlifting (Upper and Lower Body); Stationary Biking (High Resistance Intervals); Resistance Circuit Training'
 'Moderate Weight Training (Full Body Workout); Elliptical (Moderate Intensity); Stair Climber (Low to Moderate Intensity); Rowing Machine (High Intensity)'
 'Low-Impact Cardio (Elliptical, Cycling); Seated Weight Exercises; Resistance Bands and Cable Exercises; Rowing Machine (Low Resistance)'
 'Seated Cardio Machines (Rowing, Recumbent Bike); Water-Based Aerobics; Seated Strength Machines (Low Weights); Treadmill Walking at a Slow Pace'
 'Light Weightlifting for Major Muscle Groups; Stationary Biking (Moderate Intensity); Incline Walking on Treadmill; Leg Press and Arm Press (Low to Moderate Weights)'
 'Rowing Machine (Moderate Intensity); Resistance Training with Machines (Light Weight

In [15]:
# Encode 'BMIcase' into numerical values
label_encoder = LabelEncoder()
data['BMIcase_encoded'] = label_encoder.fit_transform(data['BMIcase'])

In [16]:
data.head()

,Weight,Height,BMI,Age,BMIcase,Exercise Intensity,Gym Exercise,Home Exercise,BMIcase_encoded
0,92.085190,1.760250,29.719488,59,over weight,5,Treadmill Walking with Intervals; Moderate Wei...,Modified Burpees; Step-Ups on a Sturdy Chair o...,4
1,61.089124,1.595499,23.997776,25,normal,4,Moderate Weight Training (Full Body Workout); ...,Jump Rope; High Knees; Planks; Squats and Lung...,2
2,82.454037,1.816538,24.987499,50,normal,4,Moderate Weight Training (Full Body Workout); ...,Jump Rope; High Knees; Planks; Squats and Lung...,2
3,101.713306,1.790696,31.720047,62,obese,6,"Low-Impact Cardio (Elliptical, Cycling); Seate...",Chair Squats; Modified Plank; Seated Leg Lifts...,3
4,62.347337,1.543259,26.178219,35,over weight,5,Treadmill Walking with Intervals; Moderate Wei...,Modified Burpees; Step-Ups on a Sturdy Chair o...,4


In [17]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2492 entries, 0 to 2491
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Weight              2492 non-null   float64
 1   Height              2492 non-null   float64
 2   BMI                 2492 non-null   float64
 3   Age                 2492 non-null   int64  
 4   BMIcase             2492 non-null   object 
 5   Exercise Intensity  2492 non-null   int64  
 6   Gym Exercise        2492 non-null   object 
 7   Home Exercise       2492 non-null   object 
 8   BMIcase_encoded     2492 non-null   int64  
dtypes: float64(3), int64(3), object(3)
memory usage: 175.3+ KB


In [5]:
# Define features (X) and target (y)
X = data[['Weight', 'Height', 'BMI', 'Age', 'BMIcase_encoded']]
y = data['Exercise Intensity']

In [6]:
# Standardize the features for KNN
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [7]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


# **Model Training**

In [8]:
# Initialize and train the KNN model
knn = KNeighborsClassifier(n_neighbors=5)  # You can adjust the number of neighbors (k) as needed
knn.fit(X_train, y_train)


KNeighborsClassifier()

In [9]:
# Making the predictions on the test set
y_pred = knn.predict(X_test)

# Training accuracy
train_accuracy = knn.score(X_train, y_train)
print("Training Accuracy: ", train_accuracy)

# Test accuracy
test_accuracy = knn.score(X_test, y_test)
print("Test Accuracy: ", test_accuracy)

# Print evaluation metrics
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


Training Accuracy:  1.0
Test Accuracy:  1.0
Classification Report:
               precision    recall  f1-score   support

           1       1.00      1.00      1.00         4
           2       1.00      1.00      1.00         5
           3       1.00      1.00      1.00        15
           4       1.00      1.00      1.00       114
           5       1.00      1.00      1.00       112
           6       1.00      1.00      1.00       101
           7       1.00      1.00      1.00       148

    accuracy                           1.00       499
   macro avg       1.00      1.00      1.00       499
weighted avg       1.00      1.00      1.00       499

Confusion Matrix:
 [[  4   0   0   0   0   0   0]
 [  0   5   0   0   0   0   0]
 [  0   0  15   0   0   0   0]
 [  0   0   0 114   0   0   0]
 [  0   0   0   0 112   0   0]
 [  0   0   0   0   0 101   0]
 [  0   0   0   0   0   0 148]]


In [20]:
# Function to get user recommendations
def get_user_recommendations_knn(weight, height, age):
    # Calculate BMI
    bmi = weight / (height ** 2)

    # Derive BMI case based on BMI value
    if bmi < 16:
        bmi_case = 'sever thinness'
    elif 16 <= bmi < 17:
        bmi_case = 'moderate thinness'
    elif 17 <= bmi < 18.5:
        bmi_case = 'mild thinness'
    elif 18.5 <= bmi < 24.9:
        bmi_case = 'normal'
    elif 25 <= bmi < 29.9:
        bmi_case = 'over weight'
    elif 30 <= bmi < 34.9:
        bmi_case = 'obese'
    else:
        bmi_case = 'severe obese'

    # Encode the derived BMI case
    bmi_case_encoded = label_encoder.transform([bmi_case])[0]

    # Standardize input features
    input_features = scaler.transform([[weight, height, bmi, age, bmi_case_encoded]])

    # Predict Exercise Intensity
    predicted_intensity = knn.predict(input_features)[0]

    # Map intensity to exercises
    gym_exercise = data.loc[data['Exercise Intensity'] == predicted_intensity, 'Gym Exercise'].iloc[0]
    home_exercise = data.loc[data['Exercise Intensity'] == predicted_intensity, 'Home Exercise'].iloc[0]

    return predicted_intensity, gym_exercise, home_exercise, bmi_case

# Example user input
predicted_intensity, gym_exercise, home_exercise, bmi_case = get_user_recommendations_knn(
    weight=60, height=1.75, age=23
)
print(f"Predicted Exercise Intensity: {predicted_intensity}\n")
print(f"BMI Case: {bmi_case}\n")
print(f"Recommended Gym Exercise: {gym_exercise}\n")
print(f"Recommended Home Exercise: {home_exercise}\n")

Predicted Exercise Intensity: 4

BMI Case: normal

Recommended Gym Exercise: Moderate Weight Training (Full Body Workout); Elliptical (Moderate Intensity); Stair Climber (Low to Moderate Intensity); Rowing Machine (High Intensity)

Recommended Home Exercise: Jump Rope; High Knees; Planks; Squats and Lunges (Bodyweight or Light Dumbbells)



/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
